In [ ]:
# notebooks/9_test_chroma_search.ipynb

# #################################################################
# Cell 1: 설정
# #################################################################
import os
import sys
sys.path.append(os.path.abspath('../src'))

from chromadb import PersistentClient
from sentence_transformers import SentenceTransformer
import torch
import pprint

from src.config import RAG_EMBEDDING_MODEL, CHROMA_DB_PATH

# --- 테스트 설정 ---
# 검색을 수행할 언어 쌍을 지정합니다. (예: 'en_ko', 'en_ja')
LANG_SUFFIX = "en_ko"

# 검색할 키워드나 문장을 입력합니다.
QUERY_TEXT = "sequencing technology"

# 몇 개의 유사한 결과를 가져올지 지정합니다.
N_RESULTS = 3
# --------------------



In [ ]:
# #################################################################
# Cell 2: 실행
# #################################################################
try:
    # 1. ChromaDB 클라이언트 연결
    chroma_client = PersistentClient(path=CHROMA_DB_PATH)
    collection_name = f"tm_{LANG_SUFFIX}"
    collection = chroma_client.get_collection(name=collection_name)
    print(f"✅ ChromaDB '{collection_name}' 컬렉션에 연결되었습니다.")

    # 2. 임베딩 모델 로드
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    embedding_model = SentenceTransformer(RAG_EMBEDDING_MODEL, device=device)
    print(f"✅ 임베딩 모델({device})을 로드했습니다.")

    # 3. 검색어(Query)를 벡터로 변환
    query_embedding = embedding_model.encode(QUERY_TEXT, convert_to_tensor=True)
    
    # 4. ChromaDB에 쿼리 실행
    print(f"\n🔍 '{QUERY_TEXT}'와(과) 유사한 문서를 검색합니다...")
    results = collection.query(
        query_embeddings=[query_embedding.tolist()],
        n_results=N_RESULTS
    )
    
    # 5. 결과 출력
    print("\n--- 검색 결과 ---")
    for i in range(len(results['ids'][0])):
        source_text = results['documents'][0][i]
        metadata = results['metadatas'][0][i]
        target_text = metadata.get('target_text', 'N/A')
        distance = results['distances'][0][i]
        
        print(f"\n[결과 {i+1}] (유사도 점수: {1 - distance:.2f})")
        print(f"  - Source (EN): {source_text}")
        print(f"  - Target (KO): {target_text}")
        print(f"  - 위치: {metadata.get('page_path')}")

except Exception as e:
    print(f"\n❌ 오류가 발생했습니다: {e}")
    print("   - 컬렉션 이름이 정확한지, 데이터가 정상적으로 색인되었는지 확인해주세요.")